In [ ]:
import pandas as pd

# ถ้าอยู่ใน Google Colab ให้รันก่อนเพื่ออัปโหลดไฟล์
from google.colab import files
uploaded = files.upload()

# สมมุติชื่อไฟล์ว่า patient_data.xlsx
patients = pd.read_excel('DHFcc12.xlsx')  # เปลี่ยนชื่อไฟล์ตามของคุณ


In [ ]:
import pandas as pd

# โหลดทุกชีทในไฟล์เดียว
xls = pd.ExcelFile('DHFcc12.xlsx')  # <-- แก้ชื่อไฟล์ให้ตรง

# ดึงชื่อชีททั้งหมด
sheet_names = xls.sheet_names

# อ่านและรวมทุกชีทเข้า DataFrame เดียว
df_all = pd.concat([xls.parse(sheet) for sheet in sheet_names], ignore_index=True)

# ดูผลลัพธ์
df_all.head()


In [ ]:
patients = df_all  # ใช้แทนตัวแปรเดิม


In [ ]:
df_analysis = patients[['วันเริ่มป่วย', 'วันพบผป']]
demographics = patients[['วันเริ่มป่วย', 'เพศ', 'อายุ(ปี)', 'อายุ(เดือน)', 'อาชีพ',
                         'ชื่อหมู่บ้าน', 'ตำบล', 'อำเภอ', 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล']].drop_duplicates()

df_model = pd.merge(df_analysis, demographics, on='วันเริ่มป่วย', how='left')


In [ ]:
df_model.to_excel('ข้อมูลรวม1.xlsx', index=False)

In [ ]:
['เพศ', 'อายุ(ปี)', 'อาชีพ', 'ตำบล', 'อำเภอ', 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล']


เปรียบเทียบ

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# โหลดข้อมูล
df = pd.read_excel('ข้อมูลรวม1.xlsx')

# เลือกตัวแปร
cols = ['เพศ', 'อายุ(ปี)', 'อาชีพ', 'ตำบล', 'อำเภอ', 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล']
df = df[cols].copy()

# สร้างเป้าหมาย: จำนวนผู้ป่วย (นับซ้ำในกลุ่ม)
df['นับ'] = 1
grouped = df.groupby(cols).agg({'นับ': 'sum'}).reset_index()
X = grouped[cols]
y = grouped['นับ']

# แปลงข้อมูลประเภท category ให้เป็นตัวเลข
X_encoded = X.copy()
le_dict = {}
for col in X_encoded.select_dtypes(include='object'):
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col])
    le_dict[col] = le

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)


# 🔸 Random Forest
rf_mse = mean_squared_error(y_test, rf_pred)
rf_rmse = np.sqrt(rf_mse)
rf_mae = mean_absolute_error(y_test, rf_pred)

# 🔸 Deep Learning
dl_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])
dl_model.compile(optimizer='adam', loss='mse')
dl_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
dl_pred = dl_model.predict(X_test).flatten()
mse = mean_squared_error(y_test, dl_pred)
dl_rmse = np.sqrt(mse)

dl_mae = mean_absolute_error(y_test, dl_pred)

# แสดงผลลัพธ์
print("🔹 Random Forest:")
print("  RMSE:", rf_rmse)
print("  MAE :", rf_mae)

print("\n🔹 Deep Learning:")
print("  RMSE:", dl_rmse)
print("  MAE :", dl_mae)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import xgboost as xgb

# โหลดข้อมูล
df = pd.read_excel('ข้อมูลรวม1.xlsx')

# เลือกตัวแปร
cols = ['เพศ', 'อายุ(ปี)', 'อาชีพ', 'ตำบล', 'อำเภอ', 'เดือน', 'ปี', 'ไตรมาส', 'ฤดูกาล']
df = df[cols].copy()

# สร้างเป้าหมาย: จำนวนผู้ป่วย (นับซ้ำในกลุ่ม)
df['นับ'] = 1
grouped = df.groupby(cols).agg({'นับ': 'sum'}).reset_index()
X = grouped[cols]
y = grouped['นับ']

# แปลงข้อมูลประเภท category ให้เป็นตัวเลข
X_encoded = X.copy()
le_dict = {}
for col in X_encoded.select_dtypes(include='object'):
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col])
    le_dict[col] = le

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

# Deep Learning
dl_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])
dl_model.compile(optimizer='adam', loss='mse')
dl_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
dl_pred = dl_model.predict(X_test).flatten()

# XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

# คำนวณเมตริกสำหรับแต่ละโมเดล
def print_metrics(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"🔹 {model_name}:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE : {mae:.4f}")
    print(f"  R2  : {r2:.4f}\n")

print_metrics(y_test, rf_pred, "Random Forest")
print_metrics(y_test, dl_pred, "Deep Learning")
print_metrics(y_test, xgb_pred, "XGBoost")


In [ ]:
results_df = X_test.copy()
results_df['Actual'] = y_test.values
results_df['RF_Pred'] = rf_pred
results_df['DL_Pred'] = dl_pred
results_df['XGB_Pred'] = xgb_pred

results_df.to_excel('ผลลัพธ์เปรียบเทียบโมเดล.xlsx', index=False)
print("✅ บันทึกผลลัพธ์ไว้ในไฟล์: ผลลัพธ์เปรียบเทียบโมเดล1.xlsx")


# ทำทางสถิติ

In [ ]:
import pandas as pd

df = pd.read_excel('ข้อมูลรวม1.xlsx')
print(df.columns)  # ดูชื่อคอลัมน์จริง


In [ ]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 1. โหลดข้อมูล
df = pd.read_excel('ข้อมูลรวม1.xlsx')

# 2. เปลี่ยนชื่อคอลัมน์ (ถ้ามีวงเล็บหรือช่องว่าง)
df = df.rename(columns={
    'จำนวนผู้ป่วย': 'count_patients',
    'เพศ': 'gender',
    'อายุ(ปี)': 'age_year',
    'อาชีพ': 'occupation',
    'ตำบล': 'subdistrict',
    'อำเภอ': 'district',
    'เดือน': 'month',
    'ปี': 'year',
    'ไตรมาส': 'quarter',
    'ฤดูกาล': 'season'
})

# 3. รวมกลุ่มข้อมูลและนับจำนวนผู้ป่วยในแต่ละกลุ่ม
df_count = df.groupby([
    'gender', 'age_year', 'occupation', 'subdistrict', 'district',
    'month', 'year', 'quarter', 'season'
]).size().reset_index(name='count_patients')

# 4. สร้างสูตร (formula) สำหรับ glm
formula = 'count_patients ~ gender + age_year + occupation + subdistrict + district + month + year + quarter + season'

# 5. Fit Poisson Regression
poisson_model = smf.glm(formula=formula, data=df_count, family=sm.families.Poisson()).fit()

# 6. แสดงผลสรุปโมเดล
print(poisson_model.summary())

# 7. ตรวจสอบ overdispersion
dispersion = poisson_model.deviance / poisson_model.df_resid
print(f'\nDispersion ratio: {dispersion:.2f}')

if dispersion > 1.5:
    print("\nOverdispersion detected! Fit Negative Binomial Regression instead.\n")
    nb_model = smf.glm(formula=formula, data=df_count, family=sm.families.NegativeBinomial()).fit()
    print(nb_model.summary())
else:
    print("\nNo significant overdispersion detected. Poisson model is fine.")


In [ ]:
significant_vars = poisson_model.pvalues[poisson_model.pvalues < 0.05].index.tolist()
print("ตัวแปรที่มีนัยสำคัญ (p-value < 0.05):")
print(significant_vars)


In [ ]:
significant_vars = ['occupation', 'subdistrict', 'district', 'season', 'month', 'year', 'quarter']

# แปลง categorical variables ให้เป็น C(variable)
formula_terms = []
for var in significant_vars:
    if var in ['occupation', 'subdistrict', 'district', 'season']:
        formula_terms.append(f'C({var})')
    else:
        formula_terms.append(var)

formula_significant = 'count_patients ~ ' + ' + '.join(formula_terms)

poisson_model_sig = smf.glm(formula=formula_significant, data=df_count, family=sm.families.Poisson()).fit()

print(poisson_model_sig.summary())


In [ ]:
conf_int = poisson_model_sig.conf_int()
results_df = pd.DataFrame({
    'coef': params,
    'p_value': pvalues,
    'conf_low': conf_int[0],
    'conf_high': conf_int[1]
})

# แสดงเฉพาะตัวแปรที่มีนัยสำคัญ
print(results_df[results_df['p_value'] < 0.05])


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# รวมจำนวนผู้ป่วยตามฤดู
df_season = df.groupby('season').size().reset_index(name='count_patients')

# สร้างกราฟแท่ง
plt.figure(figsize=(8,5))
sns.barplot(data=df_season, x='season', y='count_patients', palette='coolwarm')
plt.title('จำนวนผู้ป่วยไข้เลือดออกตามฤดู')
plt.xlabel('ฤดู')
plt.ylabel('จำนวนผู้ป่วย')
plt.show()
